In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from pysindy import SINDy
from sklearn.preprocessing import StandardScaler
from pysindy.optimizers import STLSQ
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix 

In [172]:
df = pd.read_csv("processed_well_data.csv")

In [173]:
print(df.isnull().sum())

Well no                    0
Dia                        0
Dev(deg)                   0
LiquidFlowrate             0
Gasflowrate                0
Area (m2)                  0
z                          0
GasDens                    0
Condesnate Presence        0
Water presence             0
LiquidDens                 0
GasVis                     0
LiqVis                     0
g (m/s2)                   0
Test Vg                    0
P/T                        0
Test status                0
Vsg                        0
Vsl                        0
Rel                        0
Reg                        0
film thickness             0
d(0,90)                    0
d(15,90)                   0
d(30,90)                   0
d(45,90)                   0
d(60,90)                   0
d(90,90)                   0
d(120,90)                  0
d(150,90)                  0
d(180,90)                  0
d(210,90)                  0
d(270,90)                  0
d(360,90)                  0
friction_facto

# Data preparation 

In [174]:
# Split features
X = df[['Dia', 'Dev(deg)','Area (m2)', 'z','GasDens','LiquidDens', 'P/T','friction_factor', 'critical_film_thickness']]
y = df['Qcr']
gsflow = df['Gasflowrate']  # This is your additional target for classification metrics

# load class labels: loaded/unloaded/near loaded
loading_class = df['Test status'].apply(
    lambda x: -1 if x == 'Unloaded' else (0 if x == 'Near L.U' else 1)).to_numpy()

# Perform the train-test split, making sure to split all targets simultaneously
X_train, X_test, y_train, y_test, gsflow_train, gsflow_test, loading_train, loading_test = train_test_split(
    X, y, gsflow, loading_class, test_size=0.20, random_state=42, stratify=loading_class
)

"""
For final evaluation of the model, we need designate test data and normalize to all training data
"""
# Scale your features and continuous target (Qcr)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
X_scaled = scaler_X.transform(X)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# t_train is just an index array for plotting
t_train = np.arange(len(y_train_scaled))

#convert to a numpy array and store test data 
loading_test = np.array(loading_test)
gsflow_test = np.array(gsflow_test)
y_test = np.array(y_test)

In [175]:
# accuracy metric and confusion matrix 
def calculate_accuracy(y_pred, gsflow, loading_actual, interval):
    loading_pred = np.where(y_pred > gsflow + interval, 1, 
                          np.where(y_pred < gsflow - interval, -1, 0))
    return accuracy_score(loading_actual, loading_pred), confusion_matrix(loading_actual, loading_pred,  labels=[-1, 0, 1])

In [176]:
# K-fold cross validation
"""
def evaluate_sindy(params, X, y, gsflow, loading_class, cv_splits=5):
    alpha, threshold, interval = params
    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)
    acc_scores = []
"""
from sklearn.model_selection import StratifiedKFold  # <-- Replace KFold with this

def evaluate_sindy(params, X, y, gsflow, loading_class, cv_splits=5):
    alpha, threshold, interval = params
    
    # Use StratifiedKFold to preserve class balance in splits
    kf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    acc_scores = []
    
    for train_idx, val_idx in kf.split(X, loading_class):  # Using validation split from training data
        # divide into trianing/validation sets
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]
        gsflow_val_cv = gsflow[val_idx]
        loading_val_cv = loading_class[val_idx]
        
        scaler_X = StandardScaler()
        X_train_cv_scaled = scaler_X.fit_transform(X_train_cv)
        X_val_cv_scaled = scaler_X.transform(X_val_cv)
        
        scaler_y = StandardScaler()
        y_train_cv_scaled = scaler_y.fit_transform(y_train_cv.reshape(-1, 1))
        
        # Define optimizer for SINDy
        optimizer = STLSQ(
            alpha=alpha,
            threshold=threshold,
            max_iter=10000,
            normalize_columns=True
        )
        
        # Train model
        model = SINDy(optimizer=optimizer)
        model.fit(X_train_cv_scaled, t=np.arange(len(y_train_cv_scaled)), 
                 x_dot=y_train_cv_scaled)
        
        # Compute performance on validation set 
        y_val_pred_scaled = model.predict(X_val_cv_scaled)
        y_val_pred_cv = scaler_y.inverse_transform(y_val_pred_scaled).flatten()
        
        acc, cm = calculate_accuracy(y_val_pred_cv, gsflow_val_cv, loading_val_cv, interval)
        acc_scores.append(acc)
    
    return np.mean(acc_scores)


In [177]:

def optimize_sindy_hyperparameters(X_train, y_train, gsflow_train, loading_train, param_grid):
    best_score = -1
    best_params = None
    
    X_train = np.array(X_train)
    y_train = np.array(y_train).flatten()
    gsflow_train = np.array(gsflow_train)
    loading_train = np.array(loading_train)
    
    print("Begin training and hyperparameter optimization...")
    for alpha in param_grid['alpha']:
        for threshold in param_grid['threshold']:
            for interval in param_grid['interval']:
                score = evaluate_sindy((alpha, threshold, interval), 
                                     X_train, y_train, gsflow_train, loading_train)
                
                if score > best_score:
                    best_score = score
                    best_params = {'alpha': alpha, 'threshold': threshold, 'interval': interval}
    
    return best_params, best_score

In [178]:
# Grid search for optimal hyperparameters 
param_grid = {
    'alpha': np.logspace(-4, 0.25, 10),      
    'threshold': np.logspace(-4, 0, 10),  
    'interval': np.linspace(0, 10, 10)  
}

#  Train model and optimize hyperparameters 
best_params, best_score = optimize_sindy_hyperparameters(
    X_train, y_train, gsflow_train, loading_train, param_grid
)

print("\nBest parameters found:")
print(best_params)
print(f"Best CV accuracy: {best_score*100:.2f}%")

# Train final model on FULL training set with best params and evaluate on the final test set
final_optimizer = STLSQ(
    alpha=best_params['alpha'],
    threshold=best_params['threshold'],
    max_iter=10000,
    normalize_columns=True
)

# Evaluate model performance 
final_model = SINDy(optimizer=final_optimizer)
final_model.fit(X_train_scaled, t=t_train, x_dot=y_train_scaled)

# Evaluate on trainin set
y_pred_train_scaled = final_model.predict(X_train_scaled)
y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled).flatten()

# Evaluate on TEST set (previously unseen data)
y_pred_test_scaled = final_model.predict(X_test_scaled)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled).flatten()

# Calculate metrics on TEST set
train_acc, train_cm = calculate_accuracy(y_pred_train, gsflow_train, loading_train, best_params['interval'])
test_acc, test_cm = calculate_accuracy(y_pred_test, gsflow_test, loading_test, best_params['interval'])

print("\n=== Final Model Performance ===")
final_model.print()
print(f"\nTraining Set Classification Accuracy: {train_acc*100:.2f}%")
print(f"Test Set Classification Accuracy: {test_acc*100:.2f}%")
print("Confusion Matrix for Test Set Classification")
print(test_cm)

Begin training and hyperparameter optimization...

Best parameters found:
{'alpha': 0.5994842503189409, 'threshold': 0.12915496650148828, 'interval': 0.0}
Best CV accuracy: 70.45%

=== Final Model Performance ===
(x0)' = 3086689209694.944 1 + 107.192 x0 + -3565097616966.648 x1 + 1.868 x2 + 0.192 x3 + 1.971 x4 + -3220096138240.641 x5 + 0.378 x6 + 180.062 x7 + -137.560 x8 + -0.318 x0^2 + 1.016 x0 x1 + -0.347 x0 x2 + -0.024 x0 x3 + 2.315 x0 x4 + -0.775 x0 x5 + 73.950 x0 x7 + -0.285 x0 x8 + -2.973 x1 x2 + 3719181391151.834 x1 x5 + -0.527 x1 x6 + -0.033 x1 x8 + 1.095 x2^2 + -4.298 x2 x4 + 0.037 x2 x5 + -0.056 x2 x6 + 2.432 x2 x7 + -0.077 x2 x8 + -0.009 x3^2 + 1.065 x3 x4 + 0.473 x3 x6 + 0.110 x3 x8 + -0.556 x4 x5 + -0.627 x4 x6 + 4.272 x4 x7 + 0.180 x4 x8 + -0.338 x5^2 + -0.362 x5 x6 + -1.832 x5 x7 + 1.268 x5 x8 + 0.042 x6 x8 + 0.217 x7 x8 + -0.749 x8^2

Training Set Classification Accuracy: 76.51%
Test Set Classification Accuracy: 61.90%
Confusion Matrix for Test Set Classification
[[10  0